# Reconnaissance de caractères et de symboles

Le projet consiste à faire l’acquisition d’images d’emballages de produits alimentaires (e.g. sacs de collations, petits déjeuner, boîte de spaghetti, sac de salade, boîte de conserve) et d’identifier les éléments suivants sur ces emballages:

- zones de texte (par exemple : la liste des ingrédients)
- zones de marquage spécifique (code barres, code QR)
- zones d’éléments graphiques (Logo, personnages (par exemple, l’ourson sur le pot de beurre d’arachides, etc.)